# 20180709 batch training experiment

In [1]:
from utils import *

import os
import datetime

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

import sys
sys.path.append("/home/dongwonshin/Desktop/Open3D/build/lib")
from py3d import *

from VolumetricVAE import *

# Volumetric AE Model training with batch

In [2]:
vol_ae = VolumetricAE(_kernel_size=3, _stride=1)
vol_ae = torch.nn.DataParallel(vol_ae, device_ids = range(torch.cuda.device_count()))
vol_ae.cuda()

optimizer = optim.Adam(vol_ae.parameters(), lr=0.0001)

class_weight = torch.Tensor(np.array(
    [0.025192871093750013,
     0.99568359375,
     0.9951199340820313,
     0.9965753173828125,
     0.998511962890625,
     0.9998382568359375,
     0.9990286254882812,
     0.9993521118164063,
     0.9994937133789062,
     0.9998776245117188,
     0.9997113037109375,
     1.0,
     0.9998117065429688,
     0.9989590454101562,
     0.9985369873046875,
     0.9993685913085938,
     1.0,
     1.0,
     0.9990896606445312,
     0.999835205078125,
     0.9960134887695312]
)).cuda()

recon_loss_func = torch.nn.CrossEntropyLoss(weight=class_weight)

In [3]:
epoches = 100
batch_size = 100
data_range = 1000

for epoch in range(epoches):
    for batch_idx in range(0,data_range,batch_size):
        
        batch_input_volume = []
        batch_target_volume = []
        for idx in range(batch_idx, batch_idx+batch_size):
            comp_pcd = read_point_cloud("/home/dongwonshin/Desktop/subvolume_dataset_making/mesh_results2/%06d_complete.ply" % idx)
            voxel_volume, color_volume = pcd2voxel(comp_pcd)
            
            input_volume = colorvol2classidvol(color_volume)
            batch_input_volume.append(input_volume)
            target_volume = colorvol2classidvol(color_volume, False)
            batch_target_volume.append(target_volume)
                    
        batch_input_volume = torch.from_numpy(np.array(batch_input_volume)).float()
        batch_target_volume = torch.from_numpy(np.array(batch_target_volume)).long()
        batch_input_volume = Variable(batch_input_volume).cuda()
        batch_target_volume = Variable(batch_target_volume).cuda()
        
        optimizer.zero_grad()
    
        batch_output_volume = vol_ae.forward(batch_input_volume)

        recon_loss = recon_loss_func(batch_output_volume, batch_target_volume)
        
        loss = recon_loss
        
        loss.backward()
        optimizer.step()

        l = loss.data[0]
    
    print(l)

    # save the intermediate result
    argmaxed = torch.argmax(batch_output_volume,1)
    output_color_volume = classidvol2colorvol(argmaxed[-1].cpu().numpy())
    viz_vvae_output(output_color_volume,'./v_ae(weight,batch)',show=False,save=True)
    

/home/dongwonshin/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1749: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/dongwonshin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(2.9684, device='cuda:0')
tensor(2.2917, device='cuda:0')
tensor(1.7853, device='cuda:0')
tensor(1.6170, device='cuda:0')
tensor(1.5183, device='cuda:0')
tensor(1.5453, device='cuda:0')
tensor(1.5051, device='cuda:0')
tensor(1.5018, device='cuda:0')
tensor(1.4973, device='cuda:0')
tensor(1.4925, device='cuda:0')
tensor(1.4867, device='cuda:0')
tensor(1.4813, device='cuda:0')
tensor(1.4786, device='cuda:0')
tensor(1.4754, device='cuda:0')
tensor(1.4727, device='cuda:0')
tensor(1.4706, device='cuda:0')
tensor(1.4684, device='cuda:0')
tensor(1.4664, device='cuda:0')
tensor(1.4646, device='cuda:0')
tensor(1.4627, device='cuda:0')
tensor(1.4613, device='cuda:0')
tensor(1.4596, device='cuda:0')
tensor(1.4578, device='cuda:0')
tensor(1.4561, device='cuda:0')
tensor(1.4541, device='cuda:0')
tensor(1.4519, device='cuda:0')
tensor(1.4499, device='cuda:0')
tensor(1.4475, device='cuda:0')
tensor(1.4445, device='cuda:0')
tensor(1.4411, device='cuda:0')
tensor(1.4370, device='cuda:0')
tensor(1